In [241]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

import seaborn as sns
sns.set_theme(style="whitegrid")

load_dotenv(override=True)
SQLALCHEMY_DATABASE_URI = os.getenv("DATABASE_URL")

# Préparation 

In [242]:
# Create an engine instance
alchemyEngine = create_engine(SQLALCHEMY_DATABASE_URI)

In [243]:
# Connect to PostgreSQL server

dbConnection = alchemyEngine.connect()

In [244]:
# Read data from PostgreSQL database table and load into a DataFrame instance

df_objet_perdu = pd.read_sql("select * from \"objet_perdu\"", dbConnection)
dbConnection.close()

In [245]:
df_objet_perdu

,id,date_heure_restitution,type,gare_origine,nature,nom_recordtype,date
0,1,2014-01-01 15:58:18,"Bagagerie: sacs, valises, cartables",Lille Europe,"Sac de voyage, sac de sport, sac à bandoulière",Objet trouvé,2013-12-31 18:22:43
1,2,NaT,Pièces d'identités et papiers personnels,Lille Europe,Carte d'abonnement,Objet trouvé,2013-12-31 17:48:55
2,3,NaT,Pièces d'identités et papiers personnels,Lille Europe,Carte d'abonnement,Objet trouvé,2013-12-31 17:45:16
3,4,NaT,Pièces d'identités et papiers personnels,Lille Europe,Carte d'abonnement,Objet trouvé,2013-12-31 17:43:11
4,5,NaT,Pièces d'identités et papiers personnels,Lille Europe,Carte d'abonnement,Objet trouvé,2013-12-31 17:37:11
...,...,...,...,...,...,...,...
30914,30915,2023-01-02 16:07:54,"Appareils électroniques, informatiques, appare...",Lille Europe,"Téléphone portable protégé (étui, coque,…)",Objet trouvé,2023-01-02 05:56:37
30915,30916,NaT,Pièces d'identités et papiers personnels,Lille Europe,"Carte d'identité, passeport, permis de conduire",Objet trouvé,2023-01-01 17:28:30
30916,30917,NaT,"Appareils électroniques, informatiques, appare...",Lille Europe,"AirPod, casque audio, écouteurs",Objet trouvé,2023-01-01 14:58:24
30917,30918,2023-01-01 07:11:10,"Bagagerie: sacs, valises, cartables",Lille Europe,Sac à main,Objet trouvé,2023-01-01 07:03:54


In [247]:
df_objet_perdu.dtypes

id                                 int64
date_heure_restitution    datetime64[ns]
type                              object
gare_origine                      object
nature                            object
nom_recordtype                    object
date                      datetime64[ns]
dtype: object

In [248]:
df_objet_perdu['Year'] = df_objet_perdu['date'].dt.year
df_objet_perdu['Month'] = df_objet_perdu['date'].dt.month
df_objet_perdu['Day'] = df_objet_perdu['date'].dt.day_of_week

In [249]:
df_objet_perdu["date"] = df_objet_perdu["date"].dt.strftime('%Y-%m-%d')

In [240]:
df_objet_perdu_groupby = df_objet_perdu.groupby(['Year', 'Month'])['Day'].count().reset_index()

In [257]:
df_objet_perdu_groupby_day = df_objet_perdu.groupby(['Year','Month','Day'])['id'].count().reset_index()
df_objet_perdu_groupby_day

,Year,Month,Day,id
0,2013,11,0,52
1,2013,11,1,53
2,2013,11,2,36
3,2013,11,3,47
4,2013,11,4,41
...,...,...,...,...
762,2022,12,6,15
763,2023,1,0,9
764,2023,1,1,7
765,2023,1,2,3


In [253]:
df_objet_perdu_by_month = df_objet_perdu_groupby.groupby(['Month'])['Day'].mean().reset_index().rename(columns={'Day':"Moyenne"})

# Moyenne par mois

In [255]:
df_objet_perdu_by_month

,Month,Moyenne
0,1,249.500000
1,2,266.222222
2,3,254.666667
3,4,275.000000
4,5,260.222222
5,6,303.777778
6,7,317.666667
7,8,276.777778
8,9,314.333333
9,10,313.333333


# Moyenne par jour

In [259]:
df_objet_perdu_by_month = df_objet_perdu_groupby_day.groupby(['Day'])['id'].mean().reset_index().rename(columns={'id':"Moyenne"})
df_objet_perdu_by_month

,Day,Moyenne
0,0,49.354545
1,1,39.918182
2,2,37.100000
3,3,42.880734
4,4,40.761468
5,5,40.688073
6,6,31.509091


In [227]:
df_objet_perdu.groupby(['Month'])['id'].count().reset_index()

,Month,id
0,1,2495
1,2,2396
2,3,2292
3,4,2200
4,5,2342
5,6,2734
6,7,2859
7,8,2491
8,9,2829
9,10,2820


# Affichez entre 2016 et 2022 la somme des objets perdus:

In [261]:
df_objet_perdu_filter_year = df_objet_perdu[(df_objet_perdu['Year']>2015) & (df_objet_perdu['Year']<2023)]

In [264]:
df_objet_perdu_filter_year.groupby(['Day'])['id'].sum().reset_index()

,Day,id
0,0,80298565
1,1,62033591
2,2,57566709
3,3,65536779
4,4,62510959
5,5,61212415
6,6,50323282
